RMR data analysis
Prepare dataset
updated: 05/25/16

In [1]:
import glob, openpyxl, pandas as pd, numpy as np
from datetime import datetime, timedelta

In [2]:
## Function to read patient's information
def get_Patient_Info(sheet,data):
    
    #print("start get patient info")
    # Create a list to save patient's information
    item = []
    
    # Read cell values in row 7 to the list
    for col in range(1,8):
        value = sheet.cell(row = 7,column = col).value
        if value is not None:
            if is_number(value):
                item.append(value)
            if (value == 'F' or value == 'M'):
                item.append(value)
            if (value == 'Male'):
                item.append('M')
            if (value == 'Female'):
                item.append('F')
          
    # Read cell values in row 8 to the list
    for col in range(1,13):
        value = sheet.cell(row = 8,column = col).value
        if value is not None:
                if is_number(value):
                    item.append(value)
   
    # Create a new list to save patient information 
    data = append_Multi_Item(data,item)
    #print(data)
    
    #print("end get patient info")
    return data

In [3]:
## Function to append multiple item in a item list 
def append_Multi_Item(data, itemList):
    #print("start append_multi_item")
    for i in range(0,(len(itemList))):
        data.append(itemList[i])
    #print("end append_multi_item")
    return data

In [4]:
## Function to check the value is numeric or not
def is_number(s):
    #print("start is_number")
    try:
        float(s)
        #print("end is_number")
        return True
    except ValueError:
        #print("end is_number")
        return False

In [5]:
## Function to get yellow region index
def get_yellow_index(sheet):
    
    #print("start get_yellow_index")
    # Create a list to save the index
    index_list = []
    
    # Save the index of the number of start line of raw data
    data_start = get_start_of_data(sheet)
    
    # Save the index of the number of end line of raw data
    data_end = get_end_of_data(sheet
                              )
    # Find the start line of yellow region
    #max_row = sheet.max_row
    exit = False
    for col in range(1,12):
        for row in range(data_start, data_end+1):
            cell = sheet.cell(row = row, column = col)
            color_index = cell.fill.start_color.index
            #if color_index == 'FFFFFF00':
                #start_raw_row = row
                #start_raw_col = col
                #exit = True
                #break
            if (color_index != '00000000'):
                start_raw_row = row
                start_raw_col = col
                exit = True
                break
        if (exit):
            break
            
    
    # Find the end line of yellow region 
    for row in range(start_raw_row, data_end+1):
        cell = sheet.cell(row = row, column = start_raw_col)
        color_index = cell.fill.start_color.index
        if (color_index == "00000000"):
            end_raw_row = row - 1
            break
        if (row == data_end):
            end_raw_row = data_end
    
    # Save start and end of yellow region to the list
    start_yellow = start_raw_row - data_start + 1 
    end_yellow = end_raw_row - data_start + 1
    
    index_list = [start_yellow, end_yellow]
    
    #print("end get_yellow_index")
    return index_list

In [6]:
def get_start_of_data(sheet):
    
    #print("start get_start_of_data")
    # Save the index of the number of start line of raw data
    for row in range(27,32):
        value = sheet.cell(row = row, column = 1).value
        if value == 'min  ':
            data_start = row + 2
            break
    #print("end get_start_of_data")
    return data_start

In [7]:
def get_end_of_data(sheet):
    
    print("start get_end_of_data")
    # Get max line
    max_row = sheet.max_row
    
    
# Save the index of the numebr of end line of raw data
    try: 
        #print(max_row)
        cell_5 = sheet.cell(row = max_row, column = 5)
        #cell_6 = sheet.cell(row = max_row, column = 6)
        if (cell_5.value is not None):
            #print(cell_6.value)
            #print(cell_5.value)
            if (is_number(cell_5.value)):
                data_end = max_row
            else:
                try:
                    cell_5__rowminusone = sheet.cell(row = max_row - 1, column = 5)
                    if (cell_5__rowminusone.value is not None): 
                        #print(cell_5__rowminusone.value)
                        if (is_number(cell_5__rowminusone.value)):
                            data_end = max_row - 1
                except IndexError:
                    #print("....indexError")
                    exit = False
                    for row in range(max_row - 80 , max_row-1):
                        #print(row)
                        try: 
                            cell = sheet.cell(row = row, column = 2)
                            if (cell.value is None):
                                #print("isnone")
                                #print(row)
                                if (is_data_end(sheet, row-1)):
                                    #print(row - 1)
                                    data_end = row - 1
                                    exit = True
                                if (exit):
                                    break
                        except IndexError:
                            #print("little indexerror")
                            data_end = row - 1
                            exit = True
                        if (exit):
                            break
        else: 
            #print("else")
            exit = False
            try:
                #rint("try/...")
                cell_5__rowminusone = sheet.cell(row = max_row - 1, column = 5)
                #print("cell5")
                if (cell_5__rowminusone.value is not None): 
                    #print(cell_5__rowminusone.value)
                    if (is_number(cell_5__rowminusone.value)):
                        data_end = max_row - 1
                else:
                    for row in range(max_row - 80 , max_row-1):
                        #print(row)
                        try: 
                            cell = sheet.cell(row = row, column = 2)
                            if (cell.value is None):
                                #print("isnone")
                                #print(row)
                                if (is_data_end(sheet, row-1)):
                                    #print(row - 1)
                                    data_end = row - 1
                                    exit = True
                                if (exit):
                                    break
                        except IndexError:
                            #print("little indexerror")
                            data_end = row - 1
                            exit = True
                        if (exit):
                            break
            except IndexError:
                #print("indexError")
                exit = False
                for row in range(max_row - 80 , max_row):
                    #print(row)
                    try: 
                        cell = sheet.cell(row = row, column = 1)
                        if (cell.value is None):
                            #print("isnone")
                            #print(row)
                            if (is_data_end(sheet, row-1)):
                                #print(row - 1)
                                data_end = row - 1
                                exit = True
                            if (exit):
                                break
                    except IndexError:
                        #print("little indexerror")
                        data_end = row - 1
                        exit = True
                    if (exit):
                        break
    except IndexError: 
        #print("big indexerror")
        exit = False
        for row in range(max_row-25, max_row):
            try:
                #print(row)
                cell = sheet.cell(row = row, column = 1)
            except IndexError: 
                data_end = row - 1
                exit = True
            if (exit):
                break
    print("end get_end_of_data")
    return data_end

In [8]:
def get_data(sheet, data, yellow, df, columns):
    
    #print("start get_data")
    # Get the start line of raw data
    data_start = get_start_of_data(sheet)
    
    # Get the end line of raw data
    data_end = get_end_of_data(sheet)
    
    # Read data in excel by row and append the data to data list and df
    for row in range(data_start, data_end + 1):
        data_copy = []
        data_copy = append_Multi_Item(data_copy, data)
        #print(data_copy)
        #print(row)
        data_excel = get_data_in_row(sheet, row)
        data_copy = append_Multi_Item(data_copy, data_excel)
        #print(data_copy)
        #print(data)
        data_copy = append_Multi_Item(data_copy, yellow)
        #print(data_copy)
        #print(data)
        df2 = pd.DataFrame([data_copy], columns = columns)
        df = df.append(df2)
    
    #print("end get_data")
    return df

In [9]:
def create_dict_from_col(key, data):
    
    dic = {key:data}
    
    return dic

In [10]:
def create_dict(key, value, index):
    
    data = []
    for i in range(1,index+1):
        data.append(value)
    
    dic = {key: data}
    
    return dic

In [11]:
def get_data_in_column(sheet, data_start, data_end, col_number):
    
    #print("start get_data_in_column")
    data = []
    #Get and save the data in column 
    for row in range(data_start, data_end + 1):
        value = sheet.cell(row = row, column = col_number).value
        data.append(value)
    #print("end get_data_in_column")
    return data

In [12]:
def get_data_in_row(sheet, row_number):
    
    #print("start get_data_in_row")
    data = []
    #Get and save the data in row
    for col in range(1, 13):
        value = sheet.cell(row = row_number, column = col).value
        if col == 4 or col == 6:
            value = data[col-2] * 1000
        data.append(value)
    #print("end get_data_in_row")
    return data

In [13]:
def is_data_end(sheet, row):
    #print("start is_data_end")
    cell_9 = sheet.cell(row = row, column = 9)
    cell_10 = sheet.cell(row = row, column = 10)
    if (is_number(cell_9.value) and is_number(cell_10.value)):
        #print("end is_data_end")
        return True
    else:
        #print("end is_data_end")
        return False

In [14]:
## Obtain File Lists
path = 'P:\\Metabolic Test\\RMR\\src\\Processed Files\\Checked\\'
all_files = glob.glob(path + "*.xlsx")

In [15]:
## Create Pandas dataframe
columns = ['Patient ID', 'Age', 'Sex','Height (in)', 'Height (cm)', 'Weight (Ib)',
           'Weight (kg)', 'Time (min)', 'VO2/kg STPD (ml/kg/m)', 'VO2 STPD (L/min)',
          '(ml/min)', 'VCO2 STPD (L/min)', '(ml/min)', 'VE STPD (L/min)', 'RQ', 
           'RR (BPM)', 'VE/VO2 (BT/ST)', 'METS', 'TM SPD(mph)', 'Start','End']
df = pd.DataFrame(columns = columns)
df

,Patient ID,Age,Sex,Height (in),Height (cm),Weight (Ib),Weight (kg),Time (min),VO2/kg STPD (ml/kg/m),VO2 STPD (L/min),...,VCO2 STPD (L/min),(ml/min),VE STPD (L/min),RQ,RR (BPM),VE/VO2 (BT/ST),METS,TM SPD(mph),Start,End


In [16]:
## Read Excel Files
for i in range(0,len(all_files)):


    # Patient ID
    idNumber = i+1
    print(i)
    print("file name")
    print(all_files[i])
    
    # Initiation of a data list for row in dataframe
    data = [idNumber]
    
    # open excel work book and sheet1
    wb = openpyxl.load_workbook(all_files[i], use_iterators= True)
    sheet_1 = wb.get_sheet_by_name(name = 'Sheet1')
    #sheet_2 = wb.get_sheet_by_name(name = 'Subject Report')
    
    # Call function to update data list with age, sex, height and weight
    data = get_Patient_Info(sheet_1, data)
    
    # Call function to get the list of start and end index of yellow region
    yellow = get_yellow_index(sheet_1)
    
    # Call function to get the data and save the data to dataframe
    df = get_data(sheet_1, data, yellow, df, columns)
    
    print(df)

0
file name
P:\Metabolic Test\RMR\src\Processed Files\Checked\Albani.Teresa_4.2.2015.xlsx
start get_end_of_data
end get_end_of_data
start get_end_of_data
end get_end_of_data
    Patient ID   Age Sex  Height (in)  Height (cm)  Weight (Ib)  Weight (kg)  \
0          1.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          1.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          1.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          1.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          1.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          1.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          1.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          1.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          1.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          1.0  46.0   F  

In [18]:
df2 = df

In [24]:
df2['Time (min)'] = [item.strip() for item in df2['Time (min)'].values]

In [25]:
df2['Time new']= pd.to_datetime(df2['Time (min)'], format="%M:%S")

ValueError: unconverted data remains: 2

In [22]:
df2

,Patient ID,Age,Sex,Height (in),Height (cm),Weight (Ib),Weight (kg),Time (min),VO2/kg STPD (ml/kg/m),VO2 STPD (L/min),...,VCO2 STPD (L/min),(ml/min),VE STPD (L/min),RQ,RR (BPM),VE/VO2 (BT/ST),METS,TM SPD(mph),Start,End
0,1.0,46.0,F,67.000000,170.179993,186.380997,84.718636,0:18,4.205394,0.356275,...,0.247425,247.424841,7.473200,0.694477,12.834225,25.997837,1.20154,0,40.0,58.0
0,1.0,46.0,F,67.000000,170.179993,186.380997,84.718636,0:30,3.374727,0.285902,...,0.201463,201.463372,5.990118,0.704658,10.335917,25.967733,0.964208,0,40.0,58.0
0,1.0,46.0,F,67.000000,170.179993,186.380997,84.718636,0:49,3.794683,0.321480,...,0.227283,227.283075,6.813195,0.706989,9.424084,26.267130,1.0842,0,40.0,58.0
0,1.0,46.0,F,67.000000,170.179993,186.380997,84.718636,1:03,3.169709,0.268533,...,0.193977,193.977356,5.767123,0.722358,8.797654,26.618103,0.905631,0,40.0,58.0
0,1.0,46.0,F,67.000000,170.179993,186.380997,84.718636,1:20,3.762461,0.318751,...,0.238344,238.344163,6.795503,0.747745,6.737788,26.423296,1.07499,0,40.0,58.0
0,1.0,46.0,F,67.000000,170.179993,186.380997,84.718636,1:35,3.324426,0.281641,...,0.211946,211.945727,5.899158,0.752539,8.102633,25.960358,0.949836,0,40.0,58.0
0,1.0,46.0,F,67.000000,170.179993,186.380997,84.718636,1:46,3.010240,0.255023,...,0.191315,191.314548,5.261397,0.750184,10.978957,25.570383,0.860069,0,40.0,58.0
0,1.0,46.0,F,67.000000,170.179993,186.380997,84.718636,2:08,3.527878,0.298877,...,0.225742,225.742131,6.074594,0.755301,8.272058,25.190744,1.00797,0,40.0,58.0
0,1.0,46.0,F,67.000000,170.179993,186.380997,84.718636,2:16,3.503185,0.296785,...,0.222011,222.010523,6.056265,0.748051,7.509387,25.291759,1.00091,0,40.0,58.0
0,1.0,46.0,F,67.000000,170.179993,186.380997,84.718636,2:32,3.069382,0.260034,...,0.189974,189.974263,5.366534,0.730575,7.532957,25.578804,0.876966,0,40.0,58.0


In [27]:
df2.to_excel('P:\\Metabolic Test\\RMR\\data\\data_all.xlsx')

In [19]:
df.to_excel('U:\\Metabolic Test\\RMR\\data\\data_debug.csv', sep=',')

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190 entries, 0 to 0
Data columns (total 21 columns):
Patient ID               190 non-null float64
Age                      190 non-null float64
Sex                      190 non-null object
Height (in)              190 non-null float64
Height (cm)              190 non-null float64
Weight (Ib)              190 non-null float64
Weight (kg)              190 non-null float64
Time (min)               190 non-null object
VO2/kg STPD (ml/kg/m)    190 non-null float64
VO2 STPD (L/min)         190 non-null float64
(ml/min)                 190 non-null float64
VCO2 STPD (L/min)        190 non-null float64
(ml/min)                 190 non-null float64
VE STPD (L/min)          190 non-null float64
RQ                       190 non-null float64
RR (BPM)                 190 non-null float64
VE/VO2 (BT/ST)           190 non-null float64
METS                     190 non-null float64
TM SPD(mph)              190 non-null float64
Start                    

In [32]:
df2 = df

In [50]:
df2['Time (min)'] = [item.strip() for item in df2['Time (min)'].values]

In [51]:
df2['Time new']= pd.to_datetime(df2['Time (min)'], format="%M:%S")


In [55]:
df2.to_excel('U:\\Metabolic Test\\RMR\\data\\data_debug3.xlsx', index = False)

In [56]:
df3 =pd.read_excel('U:\\Metabolic Test\\RMR\\data\\data_debug3.xlsx')

In [77]:
df3['Time (min)'].values[10]

'2:49'

In [64]:
df3['Time new'].values[20] -  df3['Time new'].values[10]

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'

In [78]:
from datetime import datetime, timedelta
# we specify the input and the format...
t = datetime.strptime(df3['Time (min)'].values[10],"%M:%S")
t2 = datetime.strptime(df3['Time (min)'].values[20],"%M:%S")
# ...and use datetime's hour, min and sec properties to build a timedelta
delta = timedelta(minutes=t.minute, seconds=t.second)
delta2 = timedelta(minutes=t2.minute, seconds=t2.second)


In [80]:
delta

datetime.timedelta(0, 169)

In [79]:
delta2

datetime.timedelta(0, 318)

In [81]:
delta2 - delta

datetime.timedelta(0, 149)

In [ ]:
t2.